In [1]:
import duckdb
con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/data/deadlock.db")
raw_con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/match_player_raw.duckdb")

In [2]:
con.execute("DROP TABLE IF EXISTS hero_synergy_trends")

In [3]:
from services import database_functions as dbf

dbf.create_hero_synergy_trends(con)
query = con.execute("SELECT * FROM hero_synergy_trends LIMIT 1").fetchone()
print(query)
hero_synergy_trends_col = con.execute("""
        PRAGMA table_info('hero_synergy_trends');
        """).fetchall()
print(hero_synergy_trends_col)

None
[(0, 'hero_id1', 'INTEGER', True, None, True), (1, 'hero_id2', 'INTEGER', True, None, True), (2, 'wins', 'INTEGER', False, None, False), (3, 'matches_played', 'INTEGER', False, None, False), (4, 'kills1', 'INTEGER', False, None, False), (5, 'kills2', 'INTEGER', False, None, False), (6, 'deaths1', 'INTEGER', False, None, False), (7, 'deaths2', 'INTEGER', False, None, False), (8, 'assists1', 'INTEGER', False, None, False), (9, 'assists2', 'INTEGER', False, None, False), (10, 'denies1', 'INTEGER', False, None, False), (11, 'denies2', 'INTEGER', False, None, False), (12, 'last_hits1', 'INTEGER', False, None, False), (13, 'last_hits2', 'INTEGER', False, None, False), (14, 'networth1', 'INTEGER', False, None, False), (15, 'networth2', 'INTEGER', False, None, False), (16, 'obj_damage1', 'INTEGER', False, None, False), (17, 'obj_damage2', 'INTEGER', False, None, False), (18, 'creeps1', 'INTEGER', False, None, False), (19, 'creeps2', 'INTEGER', False, None, False)]


In [4]:
query = con.execute(f"""
        SELECT * FROM hero_synergy_trends
        limit 10""").fetchdf()
print(query)

   hero_id1  hero_id2  wins  matches_played  kills1  kills2  deaths1  deaths2  \
0         1         3    26              53     374     364      327      330   
1         1         4    38              79     540     479      574      550   
2         1         6    26              62     391     420      433      489   
3         1        10    43              95     775     513      652      653   
4         1        17    26              51     347     371      399      312   
5         1        18    23              51     393     218      373      322   
6         1        19    36              80     560     685      604      582   
7         1        31    41              71     470     459      495      446   
8         1        35    25              52     397     286      374      285   
9         1        52    31              54     373     308      324      327   

   assists1  assists2  denies1  denies2  last_hits1  last_hits2  networth1  \
0       548       624      304

In [5]:
con.close()
raw_con.close()

Claude SQL to count matches in matches where all 12 players exist in player_matches_history with at least 5 matches per player.

In [38]:
import duckdb
import pandas as pd

# Connect to the database
con = duckdb.connect("data/deadlock.db")

# Query to find matches where all 12 players have player trends 
# and at least 5 references in player_matches_history
query = """
WITH match_player_counts AS (
    -- Count players per match
    SELECT 
        m.match_id,
        COUNT(DISTINCT pm.account_id) AS player_count
    FROM 
        matches m
    JOIN 
        player_matches pm ON m.match_id = pm.match_id
    GROUP BY 
        m.match_id
),
players_with_trends AS (
    -- Find players who have trends
    SELECT 
        pm.match_id,
        pm.account_id,
        CASE WHEN pt.account_id IS NOT NULL THEN 1 ELSE 0 END AS has_trend
    FROM 
        player_matches pm
    LEFT JOIN 
        player_trends pt ON pm.account_id = pt.account_id
),
players_with_history AS (
    -- Count history entries per player
    SELECT 
        pm.match_id,
        pm.account_id,
        COUNT(pmh.match_id) AS history_count
    FROM 
        player_matches pm
    LEFT JOIN 
        player_matches_history pmh ON pm.account_id = pmh.account_id
    GROUP BY 
        pm.match_id, pm.account_id
),
match_trend_stats AS (
    -- Aggregate player trend stats by match
    SELECT 
        pwt.match_id,
        SUM(pwt.has_trend) AS players_with_trends,
        COUNT(*) AS total_players
    FROM 
        players_with_trends pwt
    GROUP BY 
        pwt.match_id
),
match_history_stats AS (
    -- Aggregate player history stats by match
    SELECT 
        pwh.match_id,
        COUNT(CASE WHEN pwh.history_count >= 5 THEN 1 END) AS players_with_5plus_history,
        COUNT(*) AS total_players  
    FROM 
        players_with_history pwh
    GROUP BY 
        pwh.match_id
)
-- Final results with matches meeting all criteria
SELECT 
    COUNT(*) AS matching_matches_count,
    (SELECT COUNT(*) FROM matches) AS total_matches_count
FROM 
    match_player_counts mpc
JOIN 
    match_trend_stats mts ON mpc.match_id = mts.match_id
JOIN 
    match_history_stats mhs ON mpc.match_id = mhs.match_id
WHERE 
    mpc.player_count = 12 -- Ensure match has exactly 12 players
    AND mts.players_with_trends = 12 -- All 12 players have trends
    AND mhs.players_with_5plus_history = 12; -- All 12 players have 5+ history entries
"""

# Execute the query and get results
result = con.execute(query).fetchdf()
print(f"Matches where all 12 players have trends and 5+ history entries: {result.iloc[0, 0]} out of {result.iloc[0, 1]} total matches")

# Get a sample of these matches (first 10)
sample_query = """
WITH match_player_counts AS (
    SELECT 
        m.match_id,
        COUNT(DISTINCT pm.account_id) AS player_count
    FROM 
        matches m
    JOIN 
        player_matches pm ON m.match_id = pm.match_id
    GROUP BY 
        m.match_id
),
players_with_trends AS (
    SELECT 
        pm.match_id,
        pm.account_id,
        CASE WHEN pt.account_id IS NOT NULL THEN 1 ELSE 0 END AS has_trend
    FROM 
        player_matches pm
    LEFT JOIN 
        player_trends pt ON pm.account_id = pt.account_id
),
players_with_history AS (
    SELECT 
        pm.match_id,
        pm.account_id,
        COUNT(pmh.match_id) AS history_count
    FROM 
        player_matches pm
    LEFT JOIN 
        player_matches_history pmh ON pm.account_id = pmh.account_id
    GROUP BY 
        pm.match_id, pm.account_id
),
match_trend_stats AS (
    SELECT 
        pwt.match_id,
        SUM(pwt.has_trend) AS players_with_trends,
        COUNT(*) AS total_players
    FROM 
        players_with_trends pwt
    GROUP BY 
        pwt.match_id
),
match_history_stats AS (
    SELECT 
        pwh.match_id,
        COUNT(CASE WHEN pwh.history_count >= 5 THEN 1 END) AS players_with_5plus_history,
        COUNT(*) AS total_players  
    FROM 
        players_with_history pwh
    GROUP BY 
        pwh.match_id
),
qualifying_matches AS (
    SELECT 
        mpc.match_id
    FROM 
        match_player_counts mpc
    JOIN 
        match_trend_stats mts ON mpc.match_id = mts.match_id
    JOIN 
        match_history_stats mhs ON mpc.match_id = mhs.match_id
    WHERE 
        mpc.player_count = 12
        AND mts.players_with_trends = 12
        AND mhs.players_with_5plus_history = 12
)
SELECT 
    m.match_id,
    m.start_time,
    m.game_mode,
    m.match_mode
FROM 
    matches m
JOIN 
    qualifying_matches qm ON m.match_id = qm.match_id
ORDER BY 
    m.start_time DESC
LIMIT 10;
"""

# Print sample of qualifying matches
sample_matches = con.execute(sample_query).fetchdf()
if not sample_matches.empty:
    print("\nSample of matches meeting criteria:")
    print(sample_matches)
else:
    print("\nNo matches found meeting all criteria")

# Close the connection
con.close()

Matches where all 12 players have trends and 5+ history entries: 12690 out of 13407 total matches


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Sample of matches meeting criteria:
   match_id          start_time game_mode match_mode
0  35659956 2025-05-11 05:50:38    Normal   Unranked
1  35659817 2025-05-11 05:45:43    Normal   Unranked
2  35659653 2025-05-11 05:39:20    Normal   Unranked
3  35659402 2025-05-11 05:28:31    Normal   Unranked
4  35659009 2025-05-11 05:15:59    Normal   Unranked
5  35658812 2025-05-11 05:08:51    Normal   Unranked
6  35658760 2025-05-11 05:06:48    Normal   Unranked
7  35658665 2025-05-11 05:03:33    Normal   Unranked
8  35658327 2025-05-11 04:52:25    Normal   Unranked
9  35658119 2025-05-11 04:45:02    Normal   Unranked
